In [2]:
import datetime

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import datetime
from matplotlib.dates import DateFormatter

In [3]:
data_dt = pd.read_csv("only_dt.csv")
data_esf = pd.read_csv("esf_mag.csv")
data_ch = pd.read_csv("only_checks.csv")
data_test = pd.read_csv("test_view_10000.csv")
print(data_dt.head(20))

       dt_date        nom_razr  g32goodsnumeric g34origincountryname  \
0   2022-03-15  06649/22004631                1           КОСТА-РИКА   
1   2022-03-15  06649/22004636                1               ПАНАМА   
2   2022-03-15  06650/22003657                1              ЭКВАДОР   
3   2022-03-25  06650/22003852                1              ЭКВАДОР   
4   2022-03-28  06649/22005210                1              ЭКВАДОР   
5   2022-03-28  06649/22005211                1              ЭКВАДОР   
6   2022-03-15  06650/22003645                1              ЭКВАДОР   
7   2022-03-15  06533/22009550                1               ПАНАМА   
8   2022-03-25  06533/22010317                1               ПАНАМА   
9   2022-04-04  06649/22005588                1              ЭКВАДОР   
10  2022-03-19  06649/22004851                1              ЭКВАДОР   
11  2022-03-17  06649/22004770                1            НИКАРАГУА   
12  2022-03-11  07228/22000725                1              ЭКВ

In [76]:
def read_data_dt(data, code, company, time_start, time_end):
    """
    Функция чтения файла по ДТ
    """
    data['dt_date'] = pd.to_datetime(data['dt_date'])
    time_start = pd.to_datetime(time_start)
    time_end = pd.to_datetime(time_end)
    data = data[(data['dt_date'] >= time_start) & (data['dt_date'] <= time_end)]
    data_nw = data[(data['g32goodsnumeric'] == code) & (data['g14rbunp'] == company)]
    return data_nw


def read_data_ch(data, code, company, time_start, time_end):
    """
    Функция чтения файла по Чекам
    """
    data['issued_at'] = pd.to_datetime(data['issued_at'])
    time_start = pd.to_datetime(time_start)
    time_end = pd.to_datetime(time_end)
    data = data[(data['issued_at'] >= time_start) & (data['issued_at'] <= time_end)]
    data_nw = data[(data['gtin'] == code) & (data['unp'] == company)]
    return data_nw

In [77]:
"""
2.5 Показа объёма в количественном и стоимостном выражении, ориентируясь отдельно на
 ДТ и на чеки - столбч. диаграмма 
"""

def show_sells_by_dt(df):
    """
    Функция показа объема рынка в стоимостном выражении по ДТ
    """
    grouped_df = df.groupby('dt_date')['ТС/кг, бел руб'].sum().reset_index()
    print(grouped_df.head(20))
    plt.figure(figsize=(10, 6))
    plt.bar(grouped_df.index.strftime('%Y-%m-%d'), grouped_df.values)
    plt.xlabel('Дата')
    plt.ylabel('Сумма')
    plt.title('Суммарные продажи по дням по ДТ')
    plt.xticks(rotation=55)
    plt.show()

def show_sells_by_check(df):
    """
    Функция показа объема рынка в стоимостном выражении по Чекам
    """
    grouped_df = df.groupby('issued_at')['total_amount'].sum()
    plt.figure(figsize=(10, 6))
    plt.bar(grouped_df.index.strftime('%Y-%m-%d'), grouped_df.values)
    plt.xlabel('Дата')
    plt.ylabel('Сумма')
    plt.title('Суммарные продажи по дням по чекам')
    plt.xticks(rotation=55)
    plt.show()
    
    
def show_quantity_by_check(df):
    """
    Функция показа объема рынка в количиественном выражении по Чекам
    """
    grouped_df = df.groupby('issued_at')['position_count'].sum()
    plt.figure(figsize=(10, 6))
    plt.bar(grouped_df.index.strftime('%Y-%m-%d'), grouped_df.values)
    plt.xlabel('Дата')
    plt.ylabel('Количество')
    plt.title('Количественные  продажи по дням по чекам')
    plt.xticks(rotation=55)
    plt.show()
    
def show_quantity_by_dt(df):
    """
    Функция показа объема рынка в количиественном выражении по ДТ
    """
    grouped_df = df.groupby('dt_date')['g38netweightquantity'].sum()
    plt.figure(figsize=(10, 6))
    plt.bar(grouped_df.index.strftime('%Y-%m-%d'), grouped_df.values)
    plt.xlabel('Дата')
    plt.ylabel('Количество')
    plt.title('Количественные  продажи по дням по ДТ')
    plt.xticks(rotation=55)
    plt.show()
    
"""
2.8
Подсчет процента присутствия компании в интервале
"""
def count_interval(left, right):
    """
    Функция вычисления дельты - размерности временного интервала
    """
    if isinstance(left, datetime.datetime) & isinstance(right, datetime.datetime): # Проверка на формат datetime
        return int((right - left).days) # Вычисление лямбды
    else:
        return 0 # ERROR
        
def get_activity_percent(df, time_st, time_e):
    """
    Функия расчета отношения "торговых" дней к заданому интервалу
    """
    delta = count_interval(time_st, time_e)
    active_dates = int(df['dt_date'].nunique()) # Подсчет "торговых дней"
    return (round(active_dates / delta * 100, 1), "%")  

def get_company_info_dt(data, code, time_start, time_end):
    data['dt_date'] = pd.to_datetime(data['dt_date'])
    time_start = pd.to_datetime(time_start)
    time_end = pd.to_datetime(time_end)
    data = data[(data['dt_date'] >= time_start) & (data['dt_date'] <= time_end)]
    data_nw = data[(data['g32goodsnumeric'] == code)]
    data_nw["Процент активности"] = get_activity_percent(data_nw, time_start, time_end)
    print(data_nw.info())
    print(data_nw.head(10))
    
    
def get_percent_impact(df_dt, df_ch):
    """
    Функция расчета процента продаваемости
    """
    buy = df_dt.groupby('issued_at')['total_amount'].sum()
    sell = df_ch.groupby('issued_at')['total_amount'].sum()
    return sell / buy
    
    
    
    

In [78]:
"""
БЛОК ТЕСТ
"""



code = 1 # g32goodsnumeric

company = 799797409 # g08rbunp

time_start = "2022-03-28"
time_start = pd.to_datetime(time_start)

time_end = "2022-05-01"
time_end = pd.to_datetime(time_end)

show_sells_by_dt(data_dt)
"""ans = read_data_dt(data_dt, code, company, time_start, time_end)
show_quantity_by_dt(ans)
ans.head(10)"""

"""code = 22741 # gtin

company = 900077446 # unp

time_start = "2023-02-24"
time_start = pd.to_datetime(time_start)

time_end = "2023-03-01"
time_end = pd.to_datetime(time_end)"""


"""ans = read_data_ch(data_ch, code, company, time_start, time_end)
print(ans.head(10))
show_quantity_by_check(ans)"""



       dt_date  ТС/кг, бел руб
0   2021-12-03           90.45
1   2021-12-04           16.20
2   2021-12-10           17.28
3   2021-12-15          112.77
4   2021-12-20           19.35
5   2021-12-21           40.41
6   2021-12-25           54.72
7   2021-12-28           17.73
8   2021-12-30           38.16
9   2021-12-31           18.81
10  2022-01-02           37.44
11  2022-01-03          174.06
12  2022-01-04          112.41
13  2022-01-05          108.72
14  2022-01-06          148.41
15  2022-01-07           32.13
16  2022-01-08          215.82
17  2022-01-09          207.90
18  2022-01-10          126.54
19  2022-01-11          180.90


AttributeError: 'RangeIndex' object has no attribute 'strftime'

<Figure size 1000x600 with 0 Axes>